In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot
sns.set(rc={'figure.figsize':(11.7,8.27)})

## Reading the datasets

df1 = pd.read_csv("data/BATADAL_dataset03.csv")
df2 = pd.read_csv("data/BATADAL_dataset04.csv")
df2.columns = df2.columns.str.replace(" ", "")
dftest = pd.read_csv("data/BATADAL_test_dataset.csv")

In [65]:
# fitting a stepwise model:
from pmdarima.arima import auto_arima

stepwise_fit = auto_arima(df1['L_T1'], start_p=1, start_q=1, max_p=10, max_q=10, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()



Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=28103.160, BIC=28117.316, Fit time=0.185 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=-558.086, BIC=-536.852, Fit time=0.506 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=16531.825, BIC=16553.059, Fit time=1.880 seconds
Fit ARIMA: order=(2, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=-13038.322, BIC=-13010.009, Fit time=1.404 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=-13266.336, BIC=-13230.946, Fit time=6.671 seconds
Fit ARIMA: order=(3, 0, 2) seasonal_order=(0, 0, 0, 1); AIC=-13267.957, BIC=-13218.411, Fit time=5.514 seconds
Fit ARIMA: order=(2, 0, 2) seasonal_order=(0, 0, 0, 1); AIC=-13286.692, BIC=-13244.223, Fit time=5.166 seconds
Fit ARIMA: order=(2, 0, 3) seasonal_order=(0, 0, 0, 1); AIC=-13301.240, BIC=-13251.694, Fit time=7.181 seconds
Fit ARIMA: order=(1, 0, 2

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 8761
Model:               SARIMAX(2, 0, 3)   Log Likelihood                6657.620
Date:                Thu, 30 May 2019   AIC                         -13301.240
Time:                        14:07:39   BIC                         -13251.694
Sample:                             0   HQIC                        -13284.358
                               - 8761                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0914      0.004     24.306      0.000       0.084       0.099
ar.L1          1.8316      0.009    201.424      0.000       1.814       1.849
ar.L2         -0.8658      0.009    -98.762      0.000      -0.883      -0.849
ma.L1          0.1432      0.014     10.463      0.000       0.116       0.170
ma.L2         -0.0868      0.015     -5.606      0.000      -0.117      -0.056
ma.L3         -0.0513      0.013     -3.835      0.000      -0.077      -0.025
sigma2         0.0128      0.000     79.907      0.000       0.012       0.013
===================================================================================
Ljung-Box (Q):                      478.64   Jarque-Bera (JB):               705.90
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.96   Skew:                            -0.18
Prob(H) (two-sided):                  0.21   Kurtosis:                         4.34
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [66]:
predictions = stepwise_fit.predict_in_sample()
true_data = df2[' L_T1']

fig, ax = plt.subplots()
sns.lineplot(data=predictions[0:1000], ax=ax, color='b')
sns.lineplot(data=true_data[0:1000], ax=ax, color='r')

KeyError: ' L_T1'

In [ ]:
df2.loc[abs(predictions - true_data) > 0.5, [' DATETIME', ' ATT_FLAG']]